# KSIC10차 업종에 따라 IPC 코드 검색 쿼리 생성

In [ ]:
import pandas as pd

## table 정제

In [ ]:
table = pd.read_csv('기업정보연계+KSIC_IPC_연계_final.CSV', encoding='cp949')

In [ ]:
table

,농림어업,광업,식료품 제조업,음료제조업,담배제조업,섬유제품 제조업\n:의복제외,"의복, 의복 액세서리 및 모피제품 제조업","가죽, 가방 및 신발제조업",목재 및\n나무제품제조업; 가구제외,"펄프, 종이 및\n종이제품 제조업",인쇄 및\n기록매체복제업,"코크스, 연탄 및\n석유정제품 제조업",기초 화학물질제조업,비료 및 질소 화학물 제조업,합성고무 및\n플라스틱 물질 제조업,살충제 및 기타농약제조업,"잉크, 페인트 코팅제 및 유사제품 제조업","세제, 화장품 및 광택제 제조업",그 외 기타 화학제품 제조업,화학섬유 제조업,의료용 물질 및 의약품 제조업,고무제품 및\n플라스틱 제품 제조업,비금속광물제품 제조업,1차 철강제조업,1차 비철금속 제조업,금속주조업,"구조용 금속제품,\n탱크, 증기발생기 제조업",무기 및 총포탄 제조업,기타 금속가공 제품 제조업,반도체 제조업,전자부품 제조업,컴퓨터 및 주변장치 제조업,통신 및 방송 장비 제조업,영상 및 음향기기 제조업,마그네틱 및 광학 매체 제조업,의료용 기기 제조업,"측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업","안경, 사진장비 및\n기타 광학기기 제조업",시계 및 시계부품 제조업,"전동기, 발전기 및\n전기변환, 공급, 제어장치 제조업",일차전지 및 축전지 제조업,절연선 및 케이블제조업,전구 및 조명장치 제조업,가정용 기기 제조업,기타 전기장비 제조업,사무용 기계 및\n장비 제조업,사무용 이외의 일반기계 제조업,특수 기계제조업,자동차 제조업,선박 제조업,철도장비 제조업,항공기 제조업,전투용 차량 제조업,모터사이클 제조업,그 외 분류 안된\n운송장비 제조업,가구 제조업,기타제품 제조업,"전기, 가스, 증기, 수도","하수, 폐기물처리, 원료재생, 환경복원","건설업, 건축기술 서비스","컴퓨터 프로그래밍, 정보서비스업"
0,A01B27,C22B,A21D,C12C,A24B,D04D,A41B,A43B,A44C23,B42F,B41M,C10G,C01C,C05B,C08B,A01N,C09D,A61Q,A62D,D01C,A61P,B29B,A47K,B21C,C22F,B22D,A47H,F41A,A01L,G11C,B05D,B41J2,G03H,G03B31,G11B,A61B,G01B,G02B,G04B,F99Z,H01M,H01B,B60Q,A45D,B60M,B41L,B01B,A01B(A01B27/02 제외),B60B,B60V,B60L13,B64B,F41H7,B62J,B62B,A47B,A41G,F02C,B09B,E01C,G06D
1,A01B02,E21D,A23B,C12F,A24D,D04G,A41C,A43C,B27D,D21C,B42D,C10L,C01D,C05C,C08F,A01P,NaN,C09F,B01J,D01D,C07D,B29C,B24D,C21B,C25C,NaN,B21G,F41B,A44B11,H01L,B81B,B41J27,H01P,G10L,NaN,A61C,G01C,G02C,G04C,H02B,NaN,H01H,F21H,A47G,C25F,B41J1,B01D,A01F,B60D,B63B,B61B,B64C,NaN,B62K11,B62C,A47C,A44B(A44B11 제외),F03D,B09C,E01D,G06E
2,A01C,NaN,A23C,C12G,A24F,D04H,A41D,A45C,B27K,D21H,NaN,NaN,C01F,C05D,C08G,NaN,NaN,C09G,C06B,D01F,C07J,B29D,B28B,C21C,G21H(G21H1 제외),NaN,F16T,F41C,B21D,NaN,B81C,B41J29,H01Q,H03J,NaN,A61D,G01D,G03D,G04D,H02J,NaN,H01R,F21K,A47J,G08B,G03G,B04C,A01J,B60F,B63C,B61C,B64D,NaN,B62M6,B62H,A47D,A44C(A44C23제외),H02S,B65F,E01F,G06F(G06F1 제외)
3,A01D,NaN,A23D,C12H,NaN,D06C,A41F,B68B,B27H,D21J,NaN,NaN,C01G,C05F,C08J,NaN,NaN,C11D,C06C,NaN,C07K,B29K,B28C,C21D,NaN,NaN,F17B,F41F,B21F,NaN,B82B,B41J31,H01S,H04H,NaN,A61F,G01F,G02F2,G04F,H02K,NaN,H02G,F21L,A47L,G08G,G06C,B05B,A01M,B60G,B63G,B61D,B64F,NaN,B62M7,B62K(B62K11 제외),A47F,A45B,G21H1,C02F,E02B,G06F(G06F3 제외)
4,A01G,NaN,A23F,NaN,NaN,D06J,A42B(A42B3 제외),B68C,B27M,NaN,NaN,NaN,C07B,C05G,C08K,NaN,NaN,D06L,C06D,NaN,C12N,B29L,B28D,C22C,NaN,NaN,F22B,F41G,B21H,NaN,B82Y,B41J32,H03B,H04N,NaN,A61G,G01H,G02F3,G04G,H02N,NaN,NaN,F21S,D06F,G10K,G06M,B23F,A21B,B60H,B63H,B61F,B64G,NaN,NaN,B62L,NaN,A45F,NaN,G21F,E02D,G06G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F02F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F16N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F16P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F26B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
table.columns = pd.Series(table.columns).str.replace('\n', ' ')

In [ ]:
table['특수 기계제조업']

0     A01B(A01B27/02 제외)
1                   A01F
2                   A01J
3                   A01M
4                   A21B
             ...        
92                  F02F
93                  F16N
94                  F16P
95                  F26B
96                  H05H
Name: 특수 기계제조업, Length: 97, dtype: object

In [ ]:
table.columns

Index(['농림어업', '광업', '식료품 제조업', '음료제조업', '담배제조업', '섬유제품 제조업 :의복제외',
       '의복, 의복 액세서리 및 모피제품 제조업', '가죽, 가방 및 신발제조업', '목재 및 나무제품제조업; 가구제외',
       '펄프, 종이 및 종이제품 제조업', '인쇄 및 기록매체복제업', '코크스, 연탄 및 석유정제품 제조업',
       '기초 화학물질제조업', '비료 및 질소 화학물 제조업', '합성고무 및 플라스틱 물질 제조업', '살충제 및 기타농약제조업',
       '잉크, 페인트 코팅제 및 유사제품 제조업', '세제, 화장품 및 광택제 제조업', '그 외 기타 화학제품 제조업',
       '화학섬유 제조업', '의료용 물질 및 의약품 제조업', '고무제품 및 플라스틱 제품 제조업', '비금속광물제품 제조업',
       '1차 철강제조업', '1차 비철금속 제조업', '금속주조업', '구조용 금속제품, 탱크, 증기발생기 제조업',
       '무기 및 총포탄 제조업', '기타 금속가공 제품 제조업', '반도체 제조업', '전자부품 제조업',
       '컴퓨터 및 주변장치 제조업', '통신 및 방송 장비 제조업', '영상 및 음향기기 제조업', '마그네틱 및 광학 매체 제조업',
       '의료용 기기 제조업', '측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업', '안경, 사진장비 및 기타 광학기기 제조업',
       '시계 및 시계부품 제조업', '전동기, 발전기 및 전기변환, 공급, 제어장치 제조업', '일차전지 및 축전지 제조업',
       '절연선 및 케이블제조업', '전구 및 조명장치 제조업', '가정용 기기 제조업', '기타 전기장비 제조업',
       '사무용 기계 및 장비 제조업', '사무용 이외의 일반기계 제조업', '특수 기계제조업', '자동차 제조업', '선박 제조업',
       '철도장비 제조업', '항공기 제조업', '전투용 차량 제조업', '모터

## query 추출

In [ ]:
def change_format(x):
    x = x.strip()
    if len(x) == 4:
        x = x + '*'
    else:
        if '/' not in x:
            x = x[:4] + '-' + x[4:].zfill(3) + '*'
        else:
            x2 = x.split('/')
            x = x2[0][:4] + '-' + x2[0][4:].zfill(3) + '/' + x2[1]
    return x

In [ ]:
def make_query(i):
    result = []
    exceptions = list(filter(lambda x: '제외' in x,table[table.columns[i]][table[table.columns[i]].notnull()])) 
    # exception 있는 코드 제외하고 나열
    for x in table[table.columns[i]][table[table.columns[i]].notnull()]:
        x = x.strip()
        if x in exceptions:
            continue
        else:
            x = change_format(x)
            result.append(x)
    # exception 있는 경우에 대해
    keys = [change_format(e.split('(')[0]) for e in exceptions]
    exc_dict = {k: [] for k in keys}
    for e in exceptions:
        exc_dict[change_format(e.split('(')[0])].append(change_format(e.split('(')[1].replace(')','').replace('제외','')))
    for k, v in exc_dict.items():
        result.append('({} not ({}))'.format(k, ' or '.join(v)))
    # 최종 query
    return 'IC = ({})'.format(' or '.join(result))

In [ ]:
from tqdm import tqdm

queries = {}
for i, column in tqdm(enumerate(table.columns)):
    query = make_query(i)
    queries[column] = query

61it [00:00, 1490.65it/s]


In [ ]:
queries

{'1차 비철금속 제조업': 'IC = (C22F* or C25C* or G21H-(G21H1 제외)* or (G21H* not (G21H-001*)))',
 '1차 철강제조업': 'IC = (B21C* or C21B* or C21C* or C21D* or C22C* or F16S* or F17C*)',
 '가구 제조업': 'IC = (A47B* or A47C* or A47D* or A47F*)',
 '가정용 기기 제조업': 'IC = (A45D* or A47G* or A47J* or A47L* or D06F* or E06C* or F24B* or F24C* or H05B*)',
 '가죽, 가방 및 신발제조업': 'IC = (A43B* or A43C* or A45C* or B68B* or B68C*)',
 '건설업, 건축기술 서비스': 'IC = (E01C* or E01D* or E01F* or E02B* or E02D* or E03B* or E03C* or E03F* or E04H* or E05B* or E05C* or E06B* or E21C* or E99Z* or F42D*)',
 '고무제품 및 플라스틱 제품 제조업': 'IC = (B29B* or B29C* or B29D* or B29K* or B29L* or A42B-003* or B60C* or B65D* or B67D* or C08C* or F16L*)',
 '광업': 'IC = (C22B* or E21D*)',
 '구조용 금속제품, 탱크, 증기발생기 제조업': 'IC = (A47H* or B21G* or F16T* or F17B* or F22B* or F22G* or F24D* or F24H* or G21B* or G21C* or G21D*)',
 '그 외 기타 화학제품 제조업': 'IC = (A62D* or B01J* or C06B* or C06C* or C06D* or C06F* or C08H* or C09H* or C09J* or C09K* or C10M* or C10N* or C11B* o

# KSIC 9차 코드 정리
- 더존비즈온 데이터는 KSIC9차 업종을 가지고 있으므로 10차와 통일을 해야 IPC를 연계할 수 있음.
- KSIC 9차표는 한글명과 분류별 코드 번호를 제공하고 있음.  
- KSIC9차와 KSIC10차를 연계하기 위해 코드 번호를 통일하고자 함.  
- 중분류와 세분류가 복합적으로 들어있는 표이므로 둘을 구분하여 칼럼으로 정리함.

In [ ]:
import pandas as pd

In [ ]:
ksic9 = pd.read_csv('KSIC9차_KSIC코드.csv')

In [ ]:
ksic9['산업명'] = ksic9['산업명'].apply(lambda x: x.replace('\n', ' '))

In [ ]:
import re
p = re.compile('\S+\(') 
set(ksic9['산업분류(KSIC)'].apply(lambda x: len(p.findall(x)))) # 여러 개의 괄호가 있는 건 없다.

{1}

In [ ]:
ksic9['산업분류(KSIC)_중'] = ksic9['산업분류(KSIC)'].apply(lambda x: x.split('(')[0])

In [ ]:
ksic9['산업분류(KSIC)_중'] = ksic9['산업분류(KSIC)_중'].apply(lambda x: x.split(', '))

In [ ]:
import re
p = re.compile('\S+\(\S+\)')
ksic9['산업분류(KSIC)_세1'] = ksic9['산업분류(KSIC)'].apply(lambda x: p.findall(x)[0].split('(')[0])
ksic9['산업분류(KSIC)_세2'] = ksic9['산업분류(KSIC)'].apply(lambda x: p.findall(x)[0].replace(')','').split('(')[1])

In [ ]:
ksic9

,산업명,산업분류(KSIC),산업분류(KSIC)_중,산업분류(KSIC)_세1,산업분류(KSIC)_세2
0,농림어업,"A01, A02, A03(A0000)","[A01, A02, A03]",A03,A0000
1,광업,"B05, B06, B07, B08(B0000)","[B05, B06, B07, B08]",B08,B0000
2,식료품 제조업,C10(C1000),[C10],C10,C1000
3,음료제조업,C11(C1100),[C11],C11,C1100
4,담배제조업,C12(C1200),[C12],C12,C1200
...,...,...,...,...,...
56,기타제품 제조업,C33(C3300),[C33],C33,C3300
57,"전기, 가스, 증기, 수도","D35, D36(D0000)","[D35, D36]",D36,D0000
58,"하수, 폐기물처리, 원료재생, 환경복원","E37, E38, E39(E0000)","[E37, E38, E39]",E39,E0000
59,"건설업, 건축기술 서비스","F41, F42(Y0000)","[F41, F42]",F42,Y0000


In [ ]:
ksic9 = ksic9.explode('산업분류(KSIC)_중')

In [ ]:
ksic9[ksic9['산업분류(KSIC)_중']=='C20']

,산업명,산업분류(KSIC),산업분류(KSIC)_중,산업분류(KSIC)_세1,산업분류(KSIC)_세2
12,기초 화학물질제조업,C20(C2010),C20,C20,C2010
13,비료 및 질소 화학물 제조업,C20(C2020),C20,C20,C2020
14,합성고무 및 플라스틱 물질 제조업,C20(C2030),C20,C20,C2030
15,살충제 및 기타농약제조업,C20(C2041),C20,C20,C2041
16,"잉크, 페인트 코팅제 및 유사제품 제조업",C20(C2042),C20,C20,C2042
17,"세제, 화장품 및 광택제 제조업",C20(C2043),C20,C20,C2043
18,그 외 기타 화학제품 제조업,C20(C2049),C20,C20,C2049
19,화학섬유 제조업,C20(C2050),C20,C20,C2050


In [ ]:
ksic9

,산업명,산업분류(KSIC),산업분류(KSIC)_중,산업분류(KSIC)_세1,산업분류(KSIC)_세2
0,농림어업,"A01, A02, A03(A0000)",A01,A03,A0000
0,농림어업,"A01, A02, A03(A0000)",A02,A03,A0000
0,농림어업,"A01, A02, A03(A0000)",A03,A03,A0000
1,광업,"B05, B06, B07, B08(B0000)",B05,B08,B0000
1,광업,"B05, B06, B07, B08(B0000)",B06,B08,B0000
...,...,...,...,...,...
59,"건설업, 건축기술 서비스","F41, F42(Y0000)",F41,F42,Y0000
59,"건설업, 건축기술 서비스","F41, F42(Y0000)",F42,F42,Y0000
60,"컴퓨터 프로그래밍, 정보서비스업","J582, J62, J63(Z0000)",J582,J63,Z0000
60,"컴퓨터 프로그래밍, 정보서비스업","J582, J62, J63(Z0000)",J62,J63,Z0000


In [ ]:
ksic9.reset_index(drop=True, inplace=True)
ksic9['KSIC_CODE'] = ksic9['산업분류(KSIC)_중']
ksic9.loc[ksic9.duplicated('산업분류(KSIC)_중', keep=False), 'KSIC_CODE'] = ksic9.loc[ksic9.duplicated('산업분류(KSIC)_중', keep=False), '산업분류(KSIC)_세2']

In [ ]:
ksic9 = ksic9[['산업명', 'KSIC_CODE']]

# KSIC9차 - KSIC10차 연계
- KSIC9차의 경우 한글명과 위 과정을 통해 생성한 분류 코드를 가지고 있음.  
- KSIC10차와 연계하기 위해 분류 코드 및 한글명을 통일하고자 함.  
- 통일 방법은 다음과 같음.  
  1. 분류 코드 기준 통일: 대분류+중분류, 대분류+소분류, 대분류+세분류가 같은 업종 간 통일  
  1. 주어진 한글명이 동일한 업종 간 통일  
  1. 사소하게 명칭이 다른 경우 자카드 유사도 기준으로 통일  
  1. 매칭이 안 된 경우 특정 단어를 포함하는 업종 검색을 통해 통일

In [ ]:
import pandas as pd

In [ ]:
concordance = pd.read_csv('국세청업종코드-KSIC10-KSIC9_연계완_ver2.csv')

In [ ]:
concordance.shape

(2010, 29)

In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               2
KSIC10_대분류코드            2
KSIC10_대분류              0
KSIC10_중분류코드            2
KSIC10_중분류              0
KSIC10_소분류코드            2
KSIC10_소분류              0
KSIC10_세분류코드            2
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1600
세부설명                 1296
KSIC10_항목명             24
KSIC9_코드                2
KSIC9_항목명               2
dtype: int64

In [ ]:
concordance = concordance[~concordance['KSIC10_코드'].isna()]

## 코드 기준 병합

### 소분류 기준 병합

In [ ]:
concordance['KSIC_CODE'] = concordance['KSIC10_대분류코드']+concordance['KSIC10_소분류코드'].apply(lambda x: str(int(x)).zfill(2))

In [ ]:
(concordance['KSIC_CODE'] == 'J582').sum()

5

In [ ]:
concordance = concordance.merge(ksic9, how='left')

In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               0
KSIC10_대분류코드            0
KSIC10_대분류              0
KSIC10_중분류코드            0
KSIC10_중분류              0
KSIC10_소분류코드            0
KSIC10_소분류              0
KSIC10_세분류코드            0
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1598
세부설명                 1296
KSIC10_항목명             22
KSIC9_코드                0
KSIC9_항목명               0
KSIC_CODE               0
산업명                  2003
dtype: int64

In [ ]:
concordance.reset_index(drop=True, inplace=True)

### 중분류 기준 병합

In [ ]:
concordance.loc[concordance['산업명'].isna(), 'KSIC_CODE'] = concordance.loc[concordance['산업명'].isna(), 'KSIC10_대분류코드']+concordance.loc[concordance['산업명'].isna(), 'KSIC10_중분류코드'].apply(lambda x: str(int(x)).zfill(2))

In [ ]:
concordance.loc[concordance['산업명'].isna(), '산업명'] = concordance.loc[concordance['산업명'].isna(),concordance.columns[:-1]].merge(ksic9, how='left')['산업명']

In [ ]:
concordance

,일련번호,2018년 귀속(현행)업종코드,2019년 귀속(개편후)업종코드,국세청_대분류코드,국세청_대분류,국세청_중분류코드,국세청_중분류,국세청_소분류코드,국세청_소분류,국세청_세분류코드,국세청_세분류,국세청_세세분류,국세청_연계,KSIC10_코드,KSIC10_대분류코드,KSIC10_대분류,KSIC10_중분류코드,KSIC10_중분류,KSIC10_소분류코드,KSIC10_소분류,KSIC10_세분류코드,KSIC10_세분류,KSIC10_세세분류,KSIC10_연계,메인,세부설명,KSIC10_항목명,KSIC9_코드,KSIC9_항목명,KSIC_CODE,산업명
0,1.0,11000.0,11000.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1.0,1110.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,111.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1,NaN,NaN,곡물 및 기타 식량 작물 재배업,1110.0,곡물 및 기타 식량 작물 재배업,A01,농림어업
1,1.0,11000.0,11000.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1.0,1110.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,111.0,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업,1,NaN,NaN,,1140.0,기타 작물 재배업,A01,농림어업
2,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,1121.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,채소작물 재배업,1121.0,채소작물 재배업,A01,농림어업
3,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,1121.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,,1131.0,과실작물 재배업,A01,농림어업
4,2.0,11001.0,11001.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,110.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1.0,1121.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업,1,NaN,NaN,,1132.0,음료용 및 향신용 작물 재배업,A01,농림어업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,1589.0,NaN,990090.0,U,국제 및 외국기관,99.0,국제 및 외국기관,990.0,국제 및 외국기관,9900.0,국제 및 외국기관,기타 국제 및 외국기관,1.0,99009.0,U,국제 및 외국기관,99.0,국제 및 외국기관,990.0,국제 및 외국기관,9900.0,국제 및 외국기관,기타 국제 및 외국기관,1,NaN,NaN,기타 국제 및 외국기관,99009.0,기타 국제 및 외국기관,U99,NaN
2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71511.0,M,"전문, 과학 및 기술 서비스업",71.0,전문 서비스업,715.0,회사본부 및 경영 컨설팅 서비스업,7151.0,회사본부,제조업 회사본부,1,NaN,표준산업분류에 해당하는 국세청 업종코드가 없음,제조업 회사본부,71511.0,제조업 회사본부,M71,NaN
2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71519.0,M,"전문, 과학 및 기술 서비스업",71.0,전문 서비스업,715.0,회사본부 및 경영 컨설팅 서비스업,7151.0,회사본부,기타 산업 회사본부,1,NaN,표준산업분류에 해당하는 국세청 업종코드가 없음,기타 산업 회사본부,71519.0,기타 산업 회사본부,M71,NaN
2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98100.0,T,가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동,98.0,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,981.0,자가 소비를 위한 가사 생산 활동,9810.0,자가 소비를 위한 가사 생산 활동,자가 소비를 위한 가사 생산 활동,1,NaN,표준산업분류에 해당하는 국세청 업종코드가 없음,자가 소비를 위한 가사 생산 활동,98100.0,자가 소비를 위한 가사 생산 활동,T98,NaN


In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               0
KSIC10_대분류코드            0
KSIC10_대분류              0
KSIC10_중분류코드            0
KSIC10_중분류              0
KSIC10_소분류코드            0
KSIC10_소분류              0
KSIC10_세분류코드            0
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1598
세부설명                 1296
KSIC10_항목명             22
KSIC9_코드                0
KSIC9_항목명               0
KSIC_CODE               0
산업명                  1415
dtype: int64

In [ ]:
concordance.loc[concordance['산업명'].isna(), 'KSIC_CODE'].unique()

array(['C20', 'C26', 'C24', 'C25', 'C29', 'C28', 'C27', 'C31', 'N75',
       'C34', 'E36', 'L68', 'G45', 'G47', 'G46', 'H49', 'H52', 'H50',
       'H51', 'I55', 'I56', 'J58', 'J59', 'J61', 'J63', 'J62', 'J60',
       'K64', 'K65', 'K66', 'M70', 'M71', 'M72', 'M73', 'N76', 'N74',
       'O84', 'P85', 'Q86', 'Q87', 'R91', 'R90', 'S96', 'S95', 'A03',
       'S94', 'T97', 'U99', 'T98'], dtype=object)

In [ ]:
concordance.reset_index(drop=True, inplace=True)

### 세분류 기준

In [ ]:
concordance['KSIC9_코드'].isna().sum()

0

In [ ]:
for i in concordance.loc[concordance['산업명'].isna()].index:
    if pd.notnull(concordance.loc[i, 'KSIC9_코드']):
        concordance.loc[i, 'KSIC_CODE'] = concordance.loc[i, 'KSIC10_대분류코드']+str(int(concordance.loc[i, 'KSIC9_코드'])).zfill(4)

In [ ]:
set(ksic9['KSIC_CODE'])

{'A01',
 'A02',
 'A03',
 'B05',
 'B06',
 'B07',
 'B08',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C2010',
 'C2020',
 'C2030',
 'C2041',
 'C2042',
 'C2043',
 'C2049',
 'C2050',
 'C21',
 'C22',
 'C23',
 'C2410',
 'C2420',
 'C2430',
 'C2510',
 'C2520',
 'C2590',
 'C2610',
 'C2620',
 'C2630',
 'C2640',
 'C2650',
 'C2660',
 'C2710',
 'C2720',
 'C2730',
 'C2740',
 'C2810',
 'C2820',
 'C2830',
 'C2840',
 'C2850',
 'C2890',
 'C2918',
 'C2919',
 'C2920',
 'C30',
 'C3110',
 'C3120',
 'C3130',
 'C3191',
 'C3192',
 'C3199',
 'C32',
 'C33',
 'D35',
 'D36',
 'E37',
 'E38',
 'E39',
 'F41',
 'F42',
 'J582',
 'J62',
 'J63'}

In [ ]:
concordance['KSIC_CODE']

0          A01
1          A01
2          A01
3          A01
4          A01
         ...  
2003    U99009
2004    M71511
2005    M71519
2006    T98100
2007    T98200
Name: KSIC_CODE, Length: 2008, dtype: object

In [ ]:
(concordance['KSIC_CODE'] == 'C2010').sum()

0

In [ ]:
concordance.loc[concordance['산업명'].isna(), '산업명'] = concordance.loc[concordance['산업명'].isna(),concordance.columns[:-1]].merge(ksic9, how='left')['산업명']

In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               0
KSIC10_대분류코드            0
KSIC10_대분류              0
KSIC10_중분류코드            0
KSIC10_중분류              0
KSIC10_소분류코드            0
KSIC10_소분류              0
KSIC10_세분류코드            0
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1598
세부설명                 1296
KSIC10_항목명             22
KSIC9_코드                0
KSIC9_항목명               0
KSIC_CODE               0
산업명                  1415
dtype: int64

In [ ]:
concordance.reset_index(drop=True, inplace=True)

## 이름 기준 병합

### 주어진 이름만 사용

In [ ]:
concordance['산업명'].isna().sum()

1415

In [ ]:
set(concordance.loc[concordance['산업명'].isna(),concordance.columns[:-2]].merge(ksic9, how='left', left_on = 'KSIC9_항목명', right_on = '산업명')['산업명'])

{nan,
 '마그네틱 및 광학 매체 제조업',
 '모터사이클 제조업',
 '무기 및 총포탄 제조업',
 '사무용 기계 및 장비 제조업',
 '전투용 차량 제조업'}

In [ ]:
concordance.loc[concordance['산업명'].isna(), '산업명'] = list(concordance.loc[concordance['산업명'].isna(),concordance.columns[:-2]].merge(ksic9, how='left', left_on = 'KSIC9_항목명', right_on = '산업명')['산업명'])

In [ ]:
concordance['산업명'].isna().sum()

1409

In [ ]:
set(concordance.loc[concordance['산업명'].isna(),concordance.columns[:-2]].merge(ksic9, how='left', left_on = 'KSIC10_소분류', right_on = '산업명')['산업명'])

{'1차 비철금속 제조업',
 nan,
 '가정용 기기 제조업',
 '기타 전기장비 제조업',
 '반도체 제조업',
 '의료용 기기 제조업',
 '일차전지 및 축전지 제조업',
 '전구 및 조명장치 제조업',
 '철도장비 제조업',
 '합성고무 및 플라스틱 물질 제조업',
 '화학섬유 제조업'}

In [ ]:
concordance.loc[concordance['산업명'].isna(), '산업명'] = list(concordance.loc[concordance['산업명'].isna(),concordance.columns[:-2]].merge(ksic9, how='left', left_on = 'KSIC10_소분류', right_on = '산업명')['산업명'])

In [ ]:
concordance['산업명'].isna().sum()

1357

In [ ]:
set(concordance.loc[concordance['산업명'].isna(),concordance.columns[:-2]].merge(ksic9, how='left', left_on = 'KSIC10_세분류', right_on = '산업명')['산업명'])

{nan, '그 외 기타 화학제품 제조업', '세제, 화장품 및 광택제 제조업'}

In [ ]:
concordance.loc[concordance['산업명'].isna(), '산업명'] = list(concordance.loc[concordance['산업명'].isna(),concordance.columns[:-2]].merge(ksic9, how='left', left_on = 'KSIC10_세분류', right_on = '산업명')['산업명'])

In [ ]:
concordance['산업명'].isna().sum()

1341

In [ ]:
set(concordance.loc[concordance['산업명'].isna(),concordance.columns[:-2]].merge(ksic9, how='left', left_on = 'KSIC10_세세분류', right_on = '산업명')['산업명'])

{nan}

### 유사도 기반 분류

In [ ]:
concordance2 = concordance.copy()
# concordance = concordance2.copy()

In [ ]:
def jaccard_similarity(str1, str2):
    intersection = len(list(set(str1).intersection(str2)))
    union = (len(set(str1)) + len(set(str2))) - intersection
    return float(intersection) / union

In [ ]:
def similarity_measure(list1, list2, print_len=15):
    from itertools import product
    items = [list1, list2]
    # items = [list(set(ksic9['산업명']) - set(concordance.loc[concordance['산업명'].isna(), '산업명'])), list(set(concordance['KSIC10_소분류']))]
    # items = [list(set(ksic9['산업명'])), list(set(concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류']))]
    print(concordance['산업명'].isna().sum())

    sim_df = pd.DataFrame()
    for i, j in list(product(*items)):
        sim = jaccard_similarity(i, j)
        sim_dict = {'KSIC9': i, 'KSIC10': j, 'Jaccard_sim': sim}
        # print(sim_dict)
        sim_df = sim_df.append(sim_dict, ignore_index=True)
    print(sim_df.sort_values('Jaccard_sim', ascending=False)[:print_len].to_markdown())
    return sim_df.sort_values('Jaccard_sim', ascending=False)

In [ ]:
sim_df = similarity_measure(list(set(ksic9['산업명']) - set(concordance['산업명'])), list(set(concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류'])))

1341
|      |   Jaccard_sim | KSIC10                                                          | KSIC9                                             |
|-----:|--------------:|:----------------------------------------------------------------|:--------------------------------------------------|
| 1287 |      1        | 기타 금속 가공제품 제조업                                       | 기타 금속가공 제품 제조업                         |
|   91 |      1        | 통신 및 방송장비 제조업                                         | 통신 및 방송 장비 제조업                          |
| 2646 |      1        | 영상 및 음향 기기 제조업                                        | 영상 및 음향기기 제조업                           |
| 2148 |      1        | 절연선 및 케이블 제조업                                         | 절연선 및 케이블제조업                            |
| 1506 |      1        | 기초 화학물질 제조업                                            | 기초 화학물질제조업                               |
| 1188 |      1        | 1차 철강 제조업                                                 | 1차 철강제조업 

In [ ]:
def change_notation(sim_df, condition, ksic9, concordance):

    import pprint

    def unify(x):
        import numpy as np
        return high_sim_dict[x] if x in high_sim_dict.keys() else np.nan

    if (type(condition) == float) | (type(condition) == int):
        sim_df2 = sim_df.loc[sim_df['Jaccard_sim'] >= condition]
        high_sim_dict = dict(zip(sim_df2['KSIC10'], sim_df2['KSIC9']))#; print(high_sim_dict)

    elif type(condition) == str:
        sim_df2 = sim_df.sort_values('Jaccard_sim', ascending=False)[:15].loc[sim_df['KSIC10'] == condition].sort_values('Jaccard_sim', ascending=True)
        high_sim_dict = dict(zip(sim_df2['KSIC10'], sim_df2['KSIC9']))#; print(high_sim_dict) # dict에서 key 하나에 덮어쓰는 거 때문에.. 가장 높은 순위에 해당하는거만 뽑으려고 위에서 sort 다시 해줌
        sim_df2 = sim_df2.sort_values('Jaccard_sim', ascending=False)

    elif type(condition) == list:
        sim_df2 = sim_df.sort_values('Jaccard_sim', ascending=False)[:15].loc[sim_df['KSIC10'].apply(lambda x: x in condition)].sort_values('Jaccard_sim', ascending=True)
        high_sim_dict = dict(zip(sim_df2['KSIC10'], sim_df2['KSIC9']))#; print(high_sim_dict)
        sim_df2 = sim_df2.sort_values('Jaccard_sim', ascending=False)
    
    pprint.pprint(high_sim_dict)
    concordance.loc[concordance['산업명'].isna(), '산업명'] = concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류'].apply(lambda x: unify(x))
    
    sim_df = pd.concat([sim_df, sim_df2]).drop_duplicates(['KSIC9', 'KSIC10'], keep=False)

    return concordance, sim_df

In [ ]:
sim_df2 = sim_df.copy()
# sim_df = sim_df2.copy()

In [ ]:
concordance, sim_df = change_notation(sim_df, 1, ksic9, concordance)

{'1차 철강 제조업': '1차 철강제조업',
 '기초 화학물질 제조업': '기초 화학물질제조업',
 '기타 금속 가공제품 제조업': '기타 금속가공 제품 제조업',
 '시계 및 시계 부품 제조업': '시계 및 시계부품 제조업',
 '영상 및 음향 기기 제조업': '영상 및 음향기기 제조업',
 '전자 부품 제조업': '전자부품 제조업',
 '절연선 및 케이블 제조업': '절연선 및 케이블제조업',
 '컴퓨터 및 주변 장치 제조업': '컴퓨터 및 주변장치 제조업',
 '통신 및 방송장비 제조업': '통신 및 방송 장비 제조업'}


In [ ]:
concordance['산업명'].isna().sum()

1247

In [ ]:
sim_df[:15]

,Jaccard_sim,KSIC10,KSIC9
493,0.944444,"전동기, 발전기 및 전기 변환ㆍ공급ㆍ제어 장치 제조업","전동기, 발전기 및 전기변환, 공급, 제어장치 제조업"
2746,0.941176,"구조용 금속제품, 탱크 및 증기발생기 제조업","구조용 금속제품, 탱크, 증기발생기 제조업"
2470,0.833333,금속 주조업,금속주조업
2956,0.750000,사진장비 및 광학 기기 제조업,"안경, 사진장비 및 기타 광학기기 제조업"
932,0.727273,특수 목적용 기계 제조업,특수 기계제조업
1819,0.714286,"측정, 시험, 항해, 제어 및 기타 정밀 기기 제조업; 광학 기기 제외","측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업"
1659,0.625000,그 외 기타 운송장비 제조업,그 외 분류 안된 운송장비 제조업
1538,0.583333,기타 화학제품 제조업,기초 화학물질제조업
1086,0.571429,기초 화학물질 제조업,비료 및 질소 화학물 제조업
263,0.562500,일반 목적용 기계 제조업,사무용 이외의 일반기계 제조업


In [ ]:
concordance, sim_df = change_notation(sim_df, 0.6, ksic9, concordance)

{'구조용 금속제품, 탱크 및 증기발생기 제조업': '구조용 금속제품, 탱크, 증기발생기 제조업',
 '그 외 기타 운송장비 제조업': '그 외 분류 안된 운송장비 제조업',
 '금속 주조업': '금속주조업',
 '사진장비 및 광학 기기 제조업': '안경, 사진장비 및 기타 광학기기 제조업',
 '전동기, 발전기 및 전기 변환ㆍ공급ㆍ제어 장치 제조업': '전동기, 발전기 및 전기변환, 공급, 제어장치 제조업',
 '측정, 시험, 항해, 제어 및 기타 정밀 기기 제조업; 광학 기기 제외': '측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업',
 '특수 목적용 기계 제조업': '특수 기계제조업'}


In [ ]:
concordance['산업명'].isna().sum()

1189

In [ ]:
sim_df = similarity_measure(list(set(ksic9['산업명']) - set(concordance['산업명'])), list(set(concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류'])))

1189
|     |   Jaccard_sim | KSIC10                               | KSIC9                         |
|----:|--------------:|:-------------------------------------|:------------------------------|
| 642 |      0.5625   | 비료, 농약 및 살균ㆍ살충제 제조업    | 살충제 및 기타농약제조업      |
| 109 |      0.5625   | 일반 목적용 기계 제조업              | 사무용 이외의 일반기계 제조업 |
| 588 |      0.5      | 항공기, 우주선 및 부품 제조업        | 항공기 제조업                 |
| 400 |      0.5      | 선박 및 보트 건조업                  | 선박 제조업                   |
| 619 |      0.454545 | 기타 화학제품 제조업                 | 항공기 제조업                 |
| 743 |      0.428571 | 기타 화학제품 제조업                 | 살충제 및 기타농약제조업      |
| 247 |      0.4      | 기타 화학제품 제조업                 | 비료 및 질소 화학물 제조업    |
| 535 |      0.4      | 연금 및 공제업                       | 항공기 제조업                 |
| 605 |      0.384615 | 일반 목적용 기계 제조업              | 항공기 제조업                 |
| 146 |      0.368421 | 비료, 농약 및 살균ㆍ살충제 제조업    | 비료 및 질소 화학물 제조업    |
| 580 |      0.363636 | 항공 화물 운송업         

In [ ]:
concordance2 = concordance.copy()
# concordance = concordance2.copy()

In [ ]:
sim_df = similarity_measure(list(set(ksic9['산업명']) - set(concordance['산업명'])), list(set(concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류'])), 30)

1189
|     |   Jaccard_sim | KSIC10                               | KSIC9                                  |
|----:|--------------:|:-------------------------------------|:---------------------------------------|
| 642 |      0.5625   | 비료, 농약 및 살균ㆍ살충제 제조업    | 살충제 및 기타농약제조업               |
| 109 |      0.5625   | 일반 목적용 기계 제조업              | 사무용 이외의 일반기계 제조업          |
| 588 |      0.5      | 항공기, 우주선 및 부품 제조업        | 항공기 제조업                          |
| 400 |      0.5      | 선박 및 보트 건조업                  | 선박 제조업                            |
| 619 |      0.454545 | 기타 화학제품 제조업                 | 항공기 제조업                          |
| 743 |      0.428571 | 기타 화학제품 제조업                 | 살충제 및 기타농약제조업               |
| 247 |      0.4      | 기타 화학제품 제조업                 | 비료 및 질소 화학물 제조업             |
| 535 |      0.4      | 연금 및 공제업                       | 항공기 제조업                          |
| 605 |      0.384615 | 일반 목적용 기계 제조업              | 항공기 제조업                          |
| 146 |      

In [ ]:
concordance, sim_df = change_notation(sim_df, ['비료, 농약 및 살균ㆍ살충제 제조업', '항공기, 우주선 및 부품 제조업', '선박 및 보트 건조업', '일반 목적용 기계 제조업'], ksic9, concordance)

{'비료, 농약 및 살균ㆍ살충제 제조업': '살충제 및 기타농약제조업',
 '선박 및 보트 건조업': '선박 제조업',
 '일반 목적용 기계 제조업': '사무용 이외의 일반기계 제조업',
 '항공기, 우주선 및 부품 제조업': '항공기 제조업'}


In [ ]:
concordance['산업명'].isna().sum()

1131

In [ ]:
sim_df = similarity_measure(list(set(ksic9['산업명']) - set(concordance['산업명'])), list(set(concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류'])), 50)

1131
|     |   Jaccard_sim | KSIC10                                       | KSIC9                                  |
|----:|--------------:|:---------------------------------------------|:---------------------------------------|
| 119 |      0.4      | 기타 화학제품 제조업                         | 비료 및 질소 화학물 제조업             |
|  45 |      0.35     | 건물ㆍ산업설비 청소 및 방제 서비스업         | 비료 및 질소 화학물 제조업             |
|  34 |      0.3125   | 소화물 전문 운송업                           | 비료 및 질소 화학물 제조업             |
|  97 |      0.285714 | 가전제품 및 정보 통신장비 소매업             | 비료 및 질소 화학물 제조업             |
|   1 |      0.285714 | 연료 소매업                                  | 비료 및 질소 화학물 제조업             |
| 102 |      0.285714 | 음료 소매업                                  | 비료 및 질소 화학물 제조업             |
|   5 |      0.28     | 영화, 비디오물, 방송 프로그램 제작 및 배급업 | 비료 및 질소 화학물 제조업             |
| 177 |      0.28     | 섬유, 의복, 신발 및 가죽제품 소매업          | 잉크, 페인트 코팅제 및 유사제품 제조업 |
|  94 |      0.277778 | 주점 및 비알코올 음료점업                   

In [ ]:
set(concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류'])

{'가구 내 고용활동',
 '가전제품 및 정보 통신장비 소매업',
 '개인 및 가정용품 수리업',
 '개인 및 가정용품 임대업',
 '거주 복지시설 운영업',
 '건물ㆍ산업설비 청소 및 방제 서비스업',
 '건축 기술, 엔지니어링 및 관련 기술 서비스업',
 '건축 자재, 철물 및 난방장치 도매업',
 '경비, 경호 및 탐정업',
 '고등 교육기관',
 '고용 알선 및 인력 공급업',
 '공영 우편업',
 '공중 보건 의료업',
 '광고업',
 '교육 지원 서비스업',
 '국제 및 외국기관',
 '그 외 기타 개인 서비스업',
 '그 외 기타 전문, 과학 및 기술 서비스업',
 '금융 지원 서비스업',
 '기계장비 및 관련 물품 도매업',
 '기타 과학기술 서비스업',
 '기타 교육기관',
 '기타 금융업',
 '기타 보건업',
 '기타 사업 지원 서비스업',
 '기타 사업지원 서비스업',
 '기타 상품 전문 소매업',
 '기타 생활용품 소매업',
 '기타 숙박업',
 '기타 운송관련 서비스업',
 '기타 전문 도매업',
 '기타 전문 서비스업',
 '기타 정보 서비스업',
 '기타 협회 및 단체',
 '기타 화학제품 제조업',
 '내륙 수상 및 항만 내 운송업',
 '노동조합',
 '도로 화물 운송업',
 '도서관, 사적지 및 유사 여가관련 서비스업',
 '라디오 방송업',
 '모터사이클 및 부품 판매업',
 '무점포 소매업',
 '무형 재산권 임대업',
 '문화, 오락 및 여가 용품 소매업',
 '미용, 욕탕 및 유사 서비스업',
 '법무관련 서비스업',
 '병원',
 '보관 및 창고업',
 '보험 및 연금관련 서비스업',
 '보험업',
 '부동산 임대 및 공급업',
 '부동산관련 서비스업',
 '비거주 복지시설 운영업',
 '사법 및 공공 질서 행정',
 '사업시설 유지ㆍ관리 서비스업',
 '사진 촬영 및 처리업',
 '사회 및 산업정책 행정',
 '사회보장 행정',
 '산업 및 전문가 단체',
 '산업용 기계 및 장비 수리업',
 '산업용 기계 및 장비

In [ ]:
sim_df = similarity_measure(list(set(ksic9['산업명'])), list(set(concordance.loc[concordance['산업명'].isna(), 'KSIC10_소분류'])), 50)

1131
|      |   Jaccard_sim | KSIC10                                      | KSIC9                                  |
|-----:|--------------:|:--------------------------------------------|:---------------------------------------|
| 3599 |      0.818182 | 기타 화학제품 제조업                        | 그 외 기타 화학제품 제조업             |
| 4439 |      0.777778 | 기타 화학제품 제조업                        | 기타제품 제조업                        |
| 4454 |      0.666667 | 광고업                                      | 광업                                   |
| 3839 |      0.583333 | 기타 화학제품 제조업                        | 기초 화학물질제조업                    |
| 4024 |      0.555556 | 자동차 판매업                               | 자동차 제조업                          |
| 4319 |      0.545455 | 기타 화학제품 제조업                        | 화학섬유 제조업                        |
| 3719 |      0.538462 | 기타 화학제품 제조업                        | 기타 금속가공 제품 제조업              |
| 3941 |      0.533333 | 산업용 기계 및 장비 임대업                  | 사무용 기계 및 장비 제조업             |
| 39

In [ ]:
concordance2 = concordance.copy()
sim_df2 = sim_df.copy()

In [ ]:
concordance, sim_df = change_notation(sim_df, ['기타 화학제품 제조업', '건축 기술, 엔지니어링 및 관련 기술 서비스업', '컴퓨터 프로그래밍, 시스템 통합 및 관리업'], ksic9, concordance)

{'건축 기술, 엔지니어링 및 관련 기술 서비스업': '건설업, 건축기술 서비스',
 '기타 화학제품 제조업': '그 외 기타 화학제품 제조업',
 '컴퓨터 프로그래밍, 시스템 통합 및 관리업': '컴퓨터 프로그래밍, 정보서비스업'}


In [ ]:
concordance['산업명'].isna().sum()

1115

In [ ]:
exclude = ['광고업', '판매업', '서비스업', '도매업', '수리업', '소매업', '임대업']
print(set(concordance.loc[(concordance['산업명'].isna()) & ~(concordance['KSIC10_소분류'].str.contains('|'.join(exclude))), 'KSIC10_소분류']))
print(concordance.loc[(concordance['산업명'].isna()) & ~(concordance['KSIC10_소분류'].str.contains('|'.join(exclude))), 'KSIC10_소분류'].value_counts()[:30])

{'육상 여객 운송업', '라디오 방송업', '주점 및 비알코올 음료점업', '철도 운송업', '연금 및 공제업', '수의업', '전문 디자인업', '기타 협회 및 단체', '국제 및 외국기관', '인문 및 사회과학 연구개발업', '보관 및 창고업', '서적, 잡지 및 기타 인쇄물 출판업', '사법 및 공공 질서 행정', '부동산 임대 및 공급업', '영화, 비디오물, 방송 프로그램 제작 및 배급업', '시장 조사 및 여론 조사업', '산업 및 전문가 단체', '음식점업', '고용 알선 및 인력 공급업', '노동조합', '자가 소비를 위한 가사 생산 활동', '사회보장 행정', '기타 금융업', '외무 및 국방 행정', '공영 우편업', '은행 및 저축기관', '고등 교육기관', '파이프라인 운송업', '신탁업 및 집합 투자업', '기타 숙박업', '경비, 경호 및 탐정업', '사회 및 산업정책 행정', '내륙 수상 및 항만 내 운송업', '병원', '거주 복지시설 운영업', '해상 운송업', '오디오물 출판 및 원판 녹음업', '가구 내 고용활동', '자연과학 및 공학 연구개발업', '어로 어업', '항공 화물 운송업', '항공 여객 운송업', '전기 통신업', '텔레비전 방송업', '기타 보건업', '특수학교, 외국인학교 및 대안학교', '일반 교습학원', '도로 화물 운송업', '입법 및 일반 정부 행정', '초등 교육기관', '사진 촬영 및 처리업', '공중 보건 의료업', '중등 교육기관', '자가 소비를 위한 가사 서비스 활동', '수도업', '상품 중개업', '의원', '일반 및 생활 숙박시설 운영업', '재보험업', '기타 교육기관', '비거주 복지시설 운영업', '보험업', '소화물 전문 운송업'}
상품 중개업                        59
기타 교육기관                       39
부동산 임대 및 공급업                  28
일반 교습학원                       28
음식점업   

In [ ]:
print(concordance.loc[(concordance['산업명'].isna()) & (concordance['KSIC10_소분류'].str.contains('|'.join(exclude))), 'KSIC10_소분류'].value_counts()[:30])

생활용품 도매업                   57
기타 상품 전문 소매업               50
음ㆍ식료품 및 담배 도매업             45
기타 전문 도매업                  38
기타 생활용품 소매업                33
기타 운송관련 서비스업               25
기계장비 및 관련 물품 도매업           24
스포츠 서비스업                   23
산업용 농ㆍ축산물 및 동ㆍ식물 도매업       22
섬유, 의복, 신발 및 가죽제품 소매업      22
그 외 기타 개인 서비스업             21
기타 사업 지원 서비스업              21
유원지 및 기타 오락관련 서비스업         19
창작 및 예술관련 서비스업             17
자동차 판매업                    15
음ㆍ식료품 및 담배 소매업             15
건축 자재, 철물 및 난방장치 도매업       15
무점포 소매업                    13
문화, 오락 및 여가 용품 소매업         12
자동차 부품 및 내장품 판매업           11
종합 소매업                      9
금융 지원 서비스업                  9
미용, 욕탕 및 유사 서비스업            9
법무관련 서비스업                   8
가전제품 및 정보 통신장비 소매업          8
산업용 기계 및 장비 임대업             8
그 외 기타 전문, 과학 및 기술 서비스업     7
연료 소매업                      7
기타 과학기술 서비스업                7
도서관, 사적지 및 유사 여가관련 서비스업     7
Name: KSIC10_소분류, dtype: int64


In [ ]:
concordance['KSIC10_소분류'].nunique()

234

In [ ]:
concordance.loc[(concordance['산업명'].isna()), 'KSIC10_소분류'].nunique()

117

In [ ]:
concordance.isna().sum()

일련번호                    4
2018년 귀속(현행)업종코드       73
2019년 귀속(개편후)업종코드       4
국세청_대분류코드               4
국세청_대분류                 4
국세청_중분류코드               4
국세청_중분류                 4
국세청_소분류코드               4
국세청_소분류                 4
국세청_세분류코드               4
국세청_세분류                 4
국세청_세세분류                4
국세청_연계                  4
KSIC10_코드               0
KSIC10_대분류코드            0
KSIC10_대분류              0
KSIC10_중분류코드            0
KSIC10_중분류              0
KSIC10_소분류코드            0
KSIC10_소분류              0
KSIC10_세분류코드            0
KSIC10_세분류              0
KSIC10_세세분류             0
KSIC10_연계               0
메인                   1598
세부설명                 1296
KSIC10_항목명             22
KSIC9_코드                0
KSIC9_항목명               0
KSIC_CODE               0
산업명                  1115
dtype: int64

## 나머지

In [ ]:
set(queries.keys()) - set(concordance['산업명'].unique())

{'잉크, 페인트 코팅제 및 유사제품 제조업'}

In [ ]:
concordance.loc[(concordance['KSIC10_세세분류'].str.contains('비료|질소')) & (concordance['KSIC10_소분류'] == '비료, 농약 및 살균ㆍ살충제 제조업'), '산업명'] = '비료 및 질소 화학물 제조업'

In [ ]:
concordance.loc[(concordance['KSIC10_세세분류'].str.contains('살충제|농약')) & (concordance['KSIC10_소분류'] == '비료, 농약 및 살균ㆍ살충제 제조업'), '산업명'] = '살충제 및 기타농약제조업'

In [ ]:
concordance.loc[(concordance['KSIC10_세세분류'].str.contains('잉크|페인트|코팅제')) & (concordance['KSIC10_소분류'] == '기타 화학제품 제조업'), '산업명']

339    그 외 기타 화학제품 제조업
340    그 외 기타 화학제품 제조업
341    그 외 기타 화학제품 제조업
342    그 외 기타 화학제품 제조업
Name: 산업명, dtype: object

In [ ]:
concordance.loc[(concordance['KSIC10_세세분류'].str.contains('잉크|페인트|코팅제')) & (concordance['KSIC10_소분류'] == '기타 화학제품 제조업'), '산업명'] = '잉크, 페인트 코팅제 및 유사제품 제조업'

In [ ]:
from datetime import datetime

In [ ]:
concordance.to_csv('국세청업종코드-KSIC10-KSIC9_final_{}.csv'.format(datetime.today().strftime("%m%d")))

# 최종 확인 및 테이블 생성
- 국세청-KSIC10-KSIC9-IPC를 연계한 테이블을 생성함.

In [1]:
import pandas as pd

In [67]:
concordance = pd.read_csv('국세청업종코드-KSIC10-KSIC9_final_0901.csv')

In [52]:
concordance[concordance['산업명'].isna()].value_counts('KSIC10_대분류')

KSIC10_대분류
도매 및 소매업                           466
교육 서비스업                             83
운수 및 창고업                            81
예술, 스포츠 및 여가관련 서비스업                 66
전문, 과학 및 기술 서비스업                    61
협회 및 단체, 수리 및 기타 개인 서비스업            60
사업시설 관리, 사업 지원 및 임대 서비스업            58
숙박 및 음식점업                           51
금융 및 보험업                            42
보건업 및 사회복지 서비스업                     41
정보통신업                               34
부동산업                                33
공공 행정, 국방 및 사회보장 행정                 25
제조업                                  4
농업, 임업 및 어업                          3
가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동      3
국제 및 외국기관                            2
수도, 하수 및 폐기물 처리, 원료 재생업              2
dtype: int64

In [53]:
concordance[(concordance['산업명'].isna()) & (concordance['KSIC10_대분류'] == '농업, 임업 및 어업')].value_counts('KSIC10_소분류')

KSIC10_소분류
어로 어업    3
dtype: int64

In [16]:
concordance[(concordance['산업명'].isna()) & (concordance['KSIC10_대분류'] == '제조업')].value_counts('KSIC10_소분류')

KSIC10_소분류
산업용 기계 및 장비 수리업    4
dtype: int64

In [17]:
concordance[(concordance['산업명'].isna()) & (concordance['KSIC10_대분류'] == '정보통신업')].value_counts('KSIC10_소분류')

KSIC10_소분류
영화, 비디오물, 방송 프로그램 제작 및 배급업            9
서적, 잡지 및 기타 인쇄물 출판업                   7
전기 통신업                                5
텔레비전 방송업                              4
오디오물 출판 및 원판 녹음업                      3
기타 정보 서비스업                            3
자료 처리, 호스팅, 포털 및 기타 인터넷 정보 매개 서비스업    1
라디오 방송업                               1
공영 우편업                                1
dtype: int64

In [18]:
concordance[(concordance['산업명'].isna()) & (concordance['KSIC10_대분류'] == '수도, 하수 및 폐기물 처리, 원료 재생업')].value_counts('KSIC10_소분류')

KSIC10_소분류
수도업    2
dtype: int64

In [68]:
# concordance = concordance.loc[~concordance['산업명'].isna(), ['KSIC10_대분류', '산업명']]
concordance = concordance.loc[~concordance['산업명'].isna()]

In [33]:
concordance[['KSIC10_대분류', '산업명']]

,KSIC10_대분류,산업명
0,"농업, 임업 및 어업",농림어업
1,"농업, 임업 및 어업",농림어업
2,"농업, 임업 및 어업",농림어업
3,"농업, 임업 및 어업",농림어업
4,"농업, 임업 및 어업",농림어업
...,...,...
1959,도매 및 소매업,농림어업
1960,도매 및 소매업,농림어업
1961,도매 및 소매업,농림어업
1962,도매 및 소매업,"컴퓨터 프로그래밍, 정보서비스업"


In [35]:
concordance[['KSIC10_대분류', '산업명']].groupby(['KSIC10_대분류', '산업명']).count()[:30]

Empty DataFrame
Columns: []
Index: [(건설업, 건설업, 건축기술 서비스), (광업, 광업), (농업, 임업 및 어업, 농림어업), (도매 및 소매업, 농림어업), (도매 및 소매업, 컴퓨터 프로그래밍, 정보서비스업), (수도, 하수 및 폐기물 처리, 원료 재생업, 하수, 폐기물처리, 원료재생, 환경복원), (전기, 가스, 증기 및 공기 조절 공급업, 전기, 가스, 증기, 수도), (전문, 과학 및 기술 서비스업, 건설업, 건축기술 서비스), (정보통신업, 컴퓨터 프로그래밍, 정보서비스업), (제조업, 1차 비철금속 제조업), (제조업, 1차 철강제조업), (제조업, 가구 제조업), (제조업, 가정용 기기 제조업), (제조업, 가죽, 가방 및 신발제조업), (제조업, 고무제품 및 플라스틱 제품 제조업), (제조업, 구조용 금속제품, 탱크, 증기발생기 제조업), (제조업, 그 외 기타 화학제품 제조업), (제조업, 그 외 분류 안된 운송장비 제조업), (제조업, 금속주조업), (제조업, 기초 화학물질제조업), (제조업, 기타 금속가공 제품 제조업), (제조업, 기타 전기장비 제조업), (제조업, 기타제품 제조업), (제조업, 담배제조업), (제조업, 마그네틱 및 광학 매체 제조업), (제조업, 모터사이클 제조업), (제조업, 목재 및 나무제품제조업; 가구제외), (제조업, 무기 및 총포탄 제조업), (제조업, 반도체 제조업), (제조업, 비금속광물제품 제조업)]

In [36]:
concordance[['KSIC10_대분류', '산업명']].groupby(['KSIC10_대분류', '산업명']).count()[31:]

Empty DataFrame
Columns: []
Index: [(제조업, 사무용 기계 및 장비 제조업), (제조업, 사무용 이외의 일반기계 제조업), (제조업, 살충제 및 기타농약제조업), (제조업, 선박 제조업), (제조업, 섬유제품 제조업 :의복제외), (제조업, 세제, 화장품 및 광택제 제조업), (제조업, 시계 및 시계부품 제조업), (제조업, 식료품 제조업), (제조업, 안경, 사진장비 및 기타 광학기기 제조업), (제조업, 영상 및 음향기기 제조업), (제조업, 음료제조업), (제조업, 의료용 기기 제조업), (제조업, 의료용 물질 및 의약품 제조업), (제조업, 의복, 의복 액세서리 및 모피제품 제조업), (제조업, 인쇄 및 기록매체복제업), (제조업, 일차전지 및 축전지 제조업), (제조업, 잉크, 페인트 코팅제 및 유사제품 제조업), (제조업, 자동차 제조업), (제조업, 전구 및 조명장치 제조업), (제조업, 전동기, 발전기 및 전기변환, 공급, 제어장치 제조업), (제조업, 전자부품 제조업), (제조업, 전투용 차량 제조업), (제조업, 절연선 및 케이블제조업), (제조업, 철도장비 제조업), (제조업, 측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업), (제조업, 컴퓨터 및 주변장치 제조업), (제조업, 코크스, 연탄 및 석유정제품 제조업), (제조업, 통신 및 방송 장비 제조업), (제조업, 특수 기계제조업), (제조업, 펄프, 종이 및 종이제품 제조업), (제조업, 합성고무 및 플라스틱 물질 제조업), (제조업, 항공기 제조업), (제조업, 화학섬유 제조업)]

In [38]:
concordance.loc[concordance['KSIC10_대분류'] == '도매 및 소매업', ['KSIC10_소분류', '산업명']]

,KSIC10_소분류,산업명
1959,자동차 판매업,농림어업
1960,자동차 판매업,농림어업
1961,자동차 판매업,농림어업
1962,무점포 소매업,"컴퓨터 프로그래밍, 정보서비스업"


In [69]:
import numpy as np

concordance.loc[concordance['KSIC10_대분류'] == '도매 및 소매업', '산업명'] = np.nan

In [70]:
concordance = concordance.loc[~concordance['산업명'].isna()]

In [ ]:
concordance[['KSIC10_소분류', '산업명']].groupby(['KSIC10_소분류', '산업명']).count().shape

(125, 0)

In [42]:
concordance[['KSIC10_소분류', '산업명']].groupby(['KSIC10_소분류', '산업명']).count()[:30]

,
KSIC10_소분류,산업명
1차 비철금속 제조업,1차 비철금속 제조업
1차 철강 제조업,1차 철강제조업
가구 제조업,가구 제조업
가정용 기기 제조업,가정용 기기 제조업
"가죽, 가방 및 유사 제품 제조업","가죽, 가방 및 신발제조업"
건물 건설업,"건설업, 건축기술 서비스"
건물설비 설치 공사업,"건설업, 건축기술 서비스"
건설장비 운영업,"건설업, 건축기술 서비스"
"건축 기술, 엔지니어링 및 관련 기술 서비스업","건설업, 건축기술 서비스"


In [43]:
concordance[['KSIC10_소분류', '산업명']].groupby(['KSIC10_소분류', '산업명']).count()[30:60]

Empty DataFrame
Columns: []
Index: [(기타 전기장비 제조업, 기타 전기장비 제조업), (기타 종이 및 판지 제품 제조업, 펄프, 종이 및 종이제품 제조업), (기타 화학제품 제조업, 그 외 기타 화학제품 제조업), (기타 화학제품 제조업, 세제, 화장품 및 광택제 제조업), (기타 화학제품 제조업, 잉크, 페인트 코팅제 및 유사제품 제조업), (나무제품 제조업, 목재 및 나무제품제조업; 가구제외), (낙농제품 및 식용 빙과류 제조업, 식료품 제조업), (내화, 비내화 요업제품 제조업, 비금속광물제품 제조업), (담배 제조업, 담배제조업), (도축, 육류 가공 및 저장 처리업, 식료품 제조업), (동물성 및 식물성 유지 제조업, 식료품 제조업), (동물용 사료 및 조제식품 제조업, 식료품 제조업), (마그네틱 및 광학 매체 제조업, 마그네틱 및 광학 매체 제조업), (모피제품 제조업, 의복, 의복 액세서리 및 모피제품 제조업), (무기 및 총포탄 제조업, 무기 및 총포탄 제조업), (반도체 제조업, 반도체 제조업), (방적 및 가공사 제조업, 섬유제품 제조업 :의복제외), (봉제의복 제조업, 의복, 의복 액세서리 및 모피제품 제조업), (비료, 농약 및 살균ㆍ살충제 제조업, 비료 및 질소 화학물 제조업), (비료, 농약 및 살균ㆍ살충제 제조업, 살충제 및 기타농약제조업), (비알코올 음료 및 얼음 제조업, 음료제조업), (비철금속 광업, 광업), (사진장비 및 광학 기기 제조업, 안경, 사진장비 및 기타 광학기기 제조업), (석유 정제품 제조업, 코크스, 연탄 및 석유정제품 제조업), (석탄 광업, 광업), (선박 및 보트 건조업, 선박 제조업), (섬유제품 염색, 정리 및 마무리 가공업, 섬유제품 제조업 :의복제외), (소프트웨어 개발 및 공급업, 컴퓨터 프로그래밍, 정보서비스업), (수렵 및 관련 서비스업, 농림어업), (수산물 가공 및 저장 처리업, 식료품 제조업)]

In [44]:
concordance[['KSIC10_소분류', '산업명']].groupby(['KSIC10_소분류', '산업명']).count()[60:100]

,
KSIC10_소분류,산업명
시계 및 시계 부품 제조업,시계 및 시계부품 제조업
"시멘트, 석회, 플라스터 및 그 제품 제조업",비금속광물제품 제조업
시설물 유지관리 공사업,"건설업, 건축기술 서비스"
신발 및 신발 부분품 제조업,"가죽, 가방 및 신발제조업"
실내건축 및 건축마무리 공사업,"건설업, 건축기술 서비스"
악기 제조업,기타제품 제조업
알코올 음료 제조업,음료제조업
양식어업 및 어업관련 서비스업,농림어업
어로 어업,농림어업


In [45]:
concordance[['KSIC10_소분류', '산업명']].groupby(['KSIC10_소분류', '산업명']).count()[100:150]

,
KSIC10_소분류,산업명
직물 직조 및 직물제품 제조업,섬유제품 제조업 :의복제외
철 광업,광업
철도장비 제조업,철도장비 제조업
축산업,농림어업
"측정, 시험, 항해, 제어 및 기타 정밀 기기 제조업; 광학 기기 제외","측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업"
컴퓨터 및 주변 장치 제조업,컴퓨터 및 주변장치 제조업
"컴퓨터 프로그래밍, 시스템 통합 및 관리업","컴퓨터 프로그래밍, 정보서비스업"
코르크 및 조물 제품 제조업,목재 및 나무제품제조업; 가구제외
코크스 및 연탄 제조업,"코크스, 연탄 및 석유정제품 제조업"


In [57]:
concordance.columns

Index(['Unnamed: 0', '일련번호', '2018년 귀속(현행)업종코드', '2019년 귀속(개편후)업종코드',
       '국세청_대분류코드', '국세청_대분류', '국세청_중분류코드', '국세청_중분류', '국세청_소분류코드', '국세청_소분류',
       '국세청_세분류코드', '국세청_세분류', '국세청_세세분류', '국세청_연계', 'KSIC10_코드',
       'KSIC10_대분류코드', 'KSIC10_대분류', 'KSIC10_중분류코드', 'KSIC10_중분류',
       'KSIC10_소분류코드', 'KSIC10_소분류', 'KSIC10_세분류코드', 'KSIC10_세분류',
       'KSIC10_세세분류', 'KSIC10_연계', '메인', '세부설명', 'KSIC10_항목명', 'KSIC9_코드',
       'KSIC9_항목명', 'KSIC_CODE', '산업명'],
      dtype='object')

In [72]:
concordance = concordance.loc[~concordance['산업명'].isna(), ['국세청_대분류코드', '국세청_대분류', '국세청_중분류코드', '국세청_중분류', '국세청_소분류코드', '국세청_소분류', 'KSIC10_코드',
       'KSIC10_대분류코드', 'KSIC10_대분류', 'KSIC10_중분류코드', 'KSIC10_중분류', 'KSIC10_소분류코드', 'KSIC10_소분류', 'KSIC9_코드', 'KSIC9_항목명', '산업명']]

In [75]:
concordance.산업명.nunique()

61

In [76]:
concordance.drop_duplicates('KSIC10_소분류', inplace=True)

```
queries = {'1차 비철금속 제조업': 'IC = (C22F* or C25C* or G21H-(G21H1 제외)* or (G21H* not (G21H-001*)))',  
 '1차 철강제조업': 'IC = (B21C* or C21B* or C21C* or C21D* or C22C* or F16S* or F17C*)',  
 '가구 제조업': 'IC = (A47B* or A47C* or A47D* or A47F*)',  
 '가정용 기기 제조업': 'IC = (A45D* or A47G* or A47J* or A47L* or D06F* or E06C* or F24B* or F24C* or H05B*)',  
 '가죽, 가방 및 신발제조업': 'IC = (A43B* or A43C* or A45C* or B68B* or B68C*)',
 '건설업, 건축기술 서비스': 'IC = (E01C* or E01D* or E01F* or E02B* or E02D* or E03B* or E03C* or E03F* or E04H* or E05B* or E05C* or E06B* or E21C* or E99Z* or F42D*)',
 '고무제품 및 플라스틱 제품 제조업': 'IC = (B29B* or B29C* or B29D* or B29K* or B29L* or A42B-003* or B60C* or B65D* or B67D* or C08C* or F16L*)',
 '광업': 'IC = (C22B* or E21D*)',
 '구조용 금속제품, 탱크, 증기발생기 제조업': 'IC = (A47H* or B21G* or F16T* or F17B* or F22B* or F22G* or F24D* or F24H* or G21B* or G21C* or G21D*)',
 '그 외 기타 화학제품 제조업': 'IC = (A62D* or B01J* or C06B* or C06C* or C06D* or C06F* or C08H* or C09H* or C09J* or C09K* or C10M* or C10N* or C11B* or C11C* or C14C* or C23F* or C23G* or C40B* or C99Z* or F42B* or (G03C* not (G03C-005* or G03C-009* or G03C-011*)))',
 '그 외 분류 안된 운송장비 제조업': 'IC = (B62B* or B62C* or B62H* or B62K-(B62K11 제외)* or B62L* or B62M-(B62M6 제외)* or B62M-(B62M7 제외)* or (B62K* not (B62K-011*)) or (B62M* not (B62M-006* or B62M-007*)))',
 '금속주조업': 'IC = (B22D*)',
 '기초 화학물질제조업': 'IC = (C01C* or C01D* or C01F* or C01G* or C07B* or C07C* or C07F* or C07G* or C07H* or C09B* or C09C* or C10B* or C10C* or C10H* or C10J* or C10K* or C25B* or G21G* or G21J*)',
 '기타 금속가공 제품 제조업': 'IC = (A01L* or A44B-011* or B21D* or B21F* or B21H* or B21J* or B21K* or B21L* or B22F* or B23B* or B23C* or B23D* or B23G* or B23H* or B25B* or B25C* or B25F* or B25G* or B25H* or B26B* or B26F* or B27B* or B27G* or C23D* or C25D* or E05D* or E05G* or F16B*)',
 '기타 전기장비 제조업': 'IC = (B60M* or C25F* or G08B* or G08G* or G10K* or H01T* or H02H* or H02M* or H05C* or H05F*)',
 '기타제품 제조업': 'IC = (A41G* or A44B-(A44B11 제외)* or A44C-(A44C23제외)* or A45B* or A45F* or A46B* or A46D* or A63B* or A63C* or A63D* or A63F* or A63G* or A63H* or A63J* or A63K* or A99Z* or B43K* or B43L* or B43M* or B44D* or B44F* or B68G* or B99Z* or D07B* or F23Q* or G09B* or G09F* or G10B* or G10C* or G10D* or G10F* or G10G* or G10H* or (A44B* not (A44B-011*)) or (A44C* not (A44C-023*)))',
 '농림어업': 'IC = (A01B-027* or A01B-002* or A01C* or A01D* or A01G* or A01H* or A01K*)',
 '담배제조업': 'IC = (A24B* or A24D* or A24F*)',
 '마그네틱 및 광학 매체 제조업': 'IC = (G11B*)',
 '모터사이클 제조업': 'IC = (B62J* or B62K-011* or B62M-006* or B62M-007*)',
 '목재 및 나무제품제조업; 가구제외': 'IC = (A44C-023* or B27D* or B27K* or B27H* or B27M* or B27N* or E04G*)',
 '무기 및 총포탄 제조업': 'IC = (F41A* or F41B* or F41C* or F41F* or F41G* or F41H-(F41H7 제외)* or F41J* or F42C* or (F41H* not (F41H-007*)))',
 '반도체 제조업': 'IC = (G11C* or H01L*)',
 '비금속광물제품 제조업': 'IC = (A47K* or B24D* or B28B* or B28C* or B28D* or B32B* or C01B* or C03B* or C03C* or C04B* or E03D* or E04B* or E04C* or E04D* or E04F*)',
 '비료 및 질소 화학물 제조업': 'IC = (C05B* or C05C* or C05D* or C05F* or C05G*)',
 '사무용 기계 및 장비 제조업': 'IC = (B41L* or B41J-001* or G03G* or G06C* or G06M* or G07B* or G07C* or G07D* or G07G*)',
 '사무용 이외의 일반기계 제조업': 'IC = (B01B* or B01D* or B04C* or B05B* or B23F* or B25D* or B41K* or B60S-003* or B65B* or B65C* or B65G* or B66B* or B66C* or B66D* or B66F* or B67C* or C10F* or C12L* or E01H* or E02C* or E21F* or F01B* or F01C* or F01D* or F01K* or F01M* or F01N* or F01P* or F02G* or F03B* or F03C* or F03G* or F04B* or F04C* or F04D* or F04F* or F15B* or F15C* or F15D* or F16C* or F16D* or F16F* or F16G* or F16H* or F16K* or F16M* or F17D* or F22-000* or F23B* or F23C* or F23D* or F23G* or F23H* or F23J* or F23K* or F23L* or F23M* or F23N* or F23R* or F24F* or F24J* or F25B* or F25C* or F25D* or F25J* or F27B* or F27D* or F28B* or F28C* or F28D* or F28F* or F28G* or G01G* or G07F* or G09D* or G09G*)',
 '살충제 및 기타농약제조업': 'IC = (A01N* or A01P*)',
 '선박 제조업': 'IC = (B60V* or B63B* or B63C* or B63G* or B63H* or B63J*)',
 '섬유제품 제조업 :의복제외': 'IC = (D04D* or D04G* or D04H* or D06C* or D06J* or D06M* or D06N* or D06P* or D06Q*)',
 '세제, 화장품 및 광택제 제조업': 'IC = (A61Q* or C09F* or C09G* or C11D* or D06L* or A61K-008*)',
 '시계 및 시계부품 제조업': 'IC = (G04B* or G04C* or G04D* or G04F* or G04G* or G04R*)',
 '식료품 제조업': 'IC = (A21D* or A23B* or A23C* or A23D* or A23F* or A23G* or A23J* or A23K* or A23L* or A23P* or C12J* or C13B* or C13K*)',
 '안경, 사진장비 및 기타 광학기기 제조업': 'IC = (G02B* or G02C* or G03D* or G02F-002* or G02F-003* or G02F-007* or G03B-(G03B31 제외)* or G03B-(G03B42 제외)* or G03C-005* or G03C-009* or G03C-011* or G03F* or (G03B* not (G03B-031* or G03B-042*)))',
 '영상 및 음향기기 제조업': 'IC = (G03B-031* or G10L* or H03J* or H04H* or H04N* or H04R*)',
 '음료제조업': 'IC = (C12C* or C12F* or C12G* or C12H*)',
 '의료용 기기 제조업': 'IC = (A61B* or A61C* or A61D* or A61F* or A61G* or A61H* or A61J* or A61L* or A61M* or A61N* or A62B* or B01L* or B04B* or C12M* or G01T* or G03B-042* or H05G*)',
 '의료용 물질 및 의약품 제조업': 'IC = (A61P* or C07D* or C07J* or C07K* or C12N* or C12P* or C12Q* or C12R* or A61K-(A61K8 제외)* or (A61K* not (A61K-008*)))',
 '의복, 의복 액세서리 및 모피제품 제조업': 'IC = (A41B* or A41C* or A41D* or A41F* or A42B-(A42B3 제외)* or A42C-005* or (A42B* not (A42B-003*)))',
 '인쇄 및 기록매체복제업': 'IC = (B41M* or B42D*)',
 '일차전지 및 축전지 제조업': 'IC = (H01M*)',
 '잉크, 페인트 코팅제 및 유사제품 제조업': 'IC = (C09D*)',
 '자동차 제조업': 'IC = (B60B* or B60D* or B60F* or B60G* or B60H* or B60J* or B60K* or B60L-(B60L13 제외)* or B60N* or B60P* or B60R* or B60S-(B60S3 제외)* or B60T* or B60W* or B62D* or E05F* or F02M* or F02N* or F02P* or F16J* or G05G* or (B60L* not (B60L-013*)) or (B60S* not (B60S-003*)))',
 '전구 및 조명장치 제조업': 'IC = (B60Q* or F21H* or F21K* or F21L* or F21S* or F21V* or F21W* or F21Y* or H01K*)',
 '전기, 가스, 증기, 수도': 'IC = (F02C* or F03D* or H02S* or G21H-001*)',
 '전동기, 발전기 및 전기변환, 공급, 제어장치 제조업': 'IC = (F99Z* or H02B* or H02J* or H02K* or H02N* or H02P*)',
 '전자부품 제조업': 'IC = (B05D* or B81B* or B81C* or B82B* or B82Y* or C30B* or G02F-001* or G12B* or H01C* or H01F* or H01G* or H01J* or H05K* or H03K* or H03L*)',
 '전투용 차량 제조업': 'IC = (F41H-007*)',
 '절연선 및 케이블제조업': 'IC = (H01B* or H01H* or H01R* or H02G*)',
 '철도장비 제조업': 'IC = (B60L-013* or B61B* or B61C* or B61D* or B61F* or B61G* or B61H* or B61J* or B61K* or B61L* or E01B*)',
 '측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업': 'IC = (G01B* or G01C* or G01D* or G01F* or G01H* or G01J* or G01K* or G01L* or G01M* or G01N* or G01P* or G01Q* or G01R* or G01S* or G01V* or G01W* or G05B* or G05D* or G05F* or G21K* or G08C*)',
 '컴퓨터 및 주변장치 제조업': 'IC = (B41J-002* or B41J-027* or B41J-029* or B41J-031* or B41J-032* or B41J-033* or B41J-035* or G06J* or G06F-001* or G06F-003* or G06K*)',
 '컴퓨터 프로그래밍, 정보서비스업': 'IC = (G06D* or G06E* or G06F-(G06F1 제외)* or G06F-(G06F3 제외)* or G06G* or G06N* or G06Q* or G06T* or G09C* or (G06F* not (G06F-001* or G06F-003*)))',
 '코크스, 연탄 및 석유정제품 제조업': 'IC = (C10G* or C10L*)',
 '통신 및 방송 장비 제조업': 'IC = (G03H* or H01P* or H01Q* or H01S* or H03B* or H03C* or H03D* or H03F* or H03G* or H03H* or H03M* or H04B* or H04J* or H04K* or H04L* or H04M* or H04Q* or H04S* or H04W* or H99Z*)',
 '특수 기계제조업': 'IC = (A01F* or A01J* or A01M* or A21B* or A21C* or A22B* or A22C* or A23N* or A24C* or A41H* or A43D* or A62C* or B01F* or B02B* or B02C* or B03B* or B03C* or B03D* or B05C* or B06B* or B07B* or B07C* or B08B* or B21B* or B22C* or B23K* or B23P* or B23Q* or B24B* or B24C* or B25J* or B26D* or B27C* or B27F* or B27J* or B27L* or B30B* or B31B* or B31C* or B31D* or B31F* or B41B* or B41C* or B41D* or B41F* or B41G* or B41N* or B42B* or B42C* or B44B* or B44C* or B65H* or B67B* or B68F* or C14B* or C23C* or D01B* or D01G* or D01H* or D02G* or D02H* or D02J* or D03C* or D03D* or D03J* or D04B* or D04C* or D05B* or D05C* or D06B* or D06G* or D06H* or D21B* or D21D* or D21F* or D21G* or D99Z* or E02F* or E21B* or F01L* or F02B* or F02D* or F02F* or F16N* or F16P* or F26B* or H05H* or (A01B* not (A01B-027/02)) or (A42C* not (A42C-005*)) or (B41J* not (B41J-001* or B41J-002* or B41J-027* or B41J-029* or B41J-031* or B41J-032* or B41J-033* or B41J-035*)))',
 '펄프, 종이 및 종이제품 제조업': 'IC = (B42F* or D21C* or D21H* or D21J*)',
 '하수, 폐기물처리, 원료재생, 환경복원': 'IC = (B09B* or B09C* or B65F* or C02F* or G21F*)',
 '합성고무 및 플라스틱 물질 제조업': 'IC = (C08B* or C08F* or C08G* or C08J* or C08K* or C08L*)',
 '항공기 제조업': 'IC = (B64B* or B64C* or B64D* or B64F* or B64G* or F02K* or F03H*)',
 '화학섬유 제조업': 'IC = (D01C* or D01D* or D01F*)'}
```



In [77]:
concordance['IPC_CODE'] = concordance['산업명'].apply(lambda x: queries[x])

In [78]:
concordance

,국세청_대분류코드,국세청_대분류,국세청_중분류코드,국세청_중분류,국세청_소분류코드,국세청_소분류,KSIC10_코드,KSIC10_대분류코드,KSIC10_대분류,KSIC10_중분류코드,KSIC10_중분류,KSIC10_소분류코드,KSIC10_소분류,KSIC9_코드,KSIC9_항목명,산업명,IPC_CODE
0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,1110.0,A,"농업, 임업 및 어업",1.0,농업,11.0,작물 재배업,1110.0,곡물 및 기타 식량 작물 재배업,농림어업,IC = (A01B-027* or A01B-002* or A01C* or A01D*...
13,A,"농업, 임업 및 어업",1.0,농업,12.0,축산업,1212.0,A,"농업, 임업 및 어업",1.0,농업,12.0,축산업,1212.0,육우 사육업,농림어업,IC = (A01B-027* or A01B-002* or A01C* or A01D*...
21,A,"농업, 임업 및 어업",1.0,농업,14.0,작물재배 및 축산 관련 서비스업,1420.0,A,"농업, 임업 및 어업",1.0,농업,14.0,작물재배 및 축산 관련 서비스업,1420.0,축산 관련 서비스업,농림어업,IC = (A01B-027* or A01B-002* or A01C* or A01D*...
23,A,"농업, 임업 및 어업",1.0,농업,14.0,작물재배 및 축산 복합농업,1300.0,A,"농업, 임업 및 어업",1.0,농업,13.0,작물재배 및 축산 복합농업,1300.0,작물재배 및 축산 복합농업,농림어업,IC = (A01B-027* or A01B-002* or A01C* or A01D*...
26,A,"농업, 임업 및 어업",1.0,농업,15.0,수렵 및 관련 서비스업,1500.0,A,"농업, 임업 및 어업",1.0,농업,15.0,수렵 및 관련 서비스업,1500.0,수렵 및 관련 서비스업,농림어업,IC = (A01B-027* or A01B-002* or A01C* or A01D*...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,F,건 설 업,45.0,전문직별 공사업,453.0,건설장비 운영업,42600.0,F,건설업,42.0,전문직별 공사업,426.0,건설장비 운영업,42500.0,건설장비 운영업,"건설업, 건축기술 서비스",IC = (E01C* or E01D* or E01F* or E02B* or E02D...
1486,J,정보통신업,64.0,정보서비스업,642.0,"자료 처리, 호스팅, 포털 및 기타 인터넷 정보 매개 서비스업",63120.0,J,정보통신업,63.0,정보서비스업,631.0,"자료 처리, 호스팅, 포털 및 기타 인터넷 정보 매개 서비스업",63120.0,포털 및 기타 인터넷 정보 매개 서비스업,"컴퓨터 프로그래밍, 정보서비스업",IC = (G06D* or G06E* or G06F-(G06F1 제외)* or G0...
1488,J,정보통신업,72.0,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",721.0,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",62021.0,J,정보통신업,62.0,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",620.0,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",62021.0,컴퓨터시스템 통합 자문 및 구축 서비스업,"컴퓨터 프로그래밍, 정보서비스업",IC = (G06D* or G06E* or G06F-(G06F1 제외)* or G0...
1490,J,정보통신업,72.0,출판업,722.0,소프트웨어 개발 및 공급업,58222.0,J,정보통신업,58.0,출판업,582.0,소프트웨어 개발 및 공급업,58222.0,응용 소프트웨어 개발 및 공급업,"컴퓨터 프로그래밍, 정보서비스업",IC = (G06D* or G06E* or G06F-(G06F1 제외)* or G0...


In [80]:
from datetime import datetime
concordance.to_csv('국세청업종코드-KSIC10-KSIC9-IPC_final_{}.csv'.format(datetime.today().strftime("%m%d")))